In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
            .appName("my_app") \
            .master("spark://spark:7077") \
            .getOrCreate()

spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://localstack:4566")
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", "foo")
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "foo")
spark._jsc.hadoopConfiguration().set("fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem")
spark._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
spark._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider","org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")

In [3]:
s3_df=spark.read.csv("s3a://mytestbucket/explain_202110291220.csv",header=True,inferSchema=True)
s3_df.show(5)

+---+-----------+-----+------+--------------------+--------------------+-------+-------------------+----+--------------------+
| id|select_type|table|  type|       possible_keys|                 key|key_len|                ref|rows|               Extra|
+---+-----------+-----+------+--------------------+--------------------+-------+-------------------+----+--------------------+
|  1|    PRIMARY|    t| range|PRIMARY,idusers,i...|idx_transaction_i...|      9|               null|4861|Using index condi...|
|  1|    PRIMARY|    u|eq_ref|             PRIMARY|             PRIMARY|      4|       mm.t.idusers|   1|                null|
|  1|    PRIMARY|    f|   ref|idtransactions,id...|      idtransactions|      4|mm.t.idtransactions|   1|         Using where|
|  1|    PRIMARY|    a|eq_ref|  PRIMARY,idairlines|             PRIMARY|      4|    mm.f.idairlines|   1|                null|
|  1|    PRIMARY|    n|eq_ref|PRIMARY,idstatus_...|             PRIMARY|      4|        mm.t.idnips|   1|      